In [64]:
from rating_tools import get_tournament_results_by_id, get_tournaments
from rating_tools import get_all_players, get_player_info, api_call

In [97]:
from collections import Counter
import numpy as np
from tqdm import tqdm_notebook
import datetime
from dateutil.parser import parse as date_parse
import math
import json

In [11]:
def get_player_rating(player_id, rating_release=None):
    if rating_release is None:
        rating_release = 'last'
    url = f"http://rating.chgk.info/api/players/{player_id}/rating/{rating_release}.json"
    return api_call(url)

In [20]:
player_ratings = dict()
for player_id in tqdm_notebook(range(220000)):
    rating_info = get_player_rating(player_id, 1433)
    if rating_info:
        player_ratings[player_id] = int(rating_info['rating'])

KeyboardInterrupt: 

In [46]:
def rolling_window(a, window):
    a = np.append(a, np.zeros(window - 1))
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [123]:
def calculate_bonus_predictions(tournament_ratings, c=1):
    tournament_ratings[::-1].sort()
    return np.round(rolling_window(tournament_ratings, 15).dot(2.**np.arange(0, -15, -1)) * c)

In [51]:
test_participants = np.random.randint(3000, 10000, 1000)

In [90]:
def get_tournaments_for_release(release_date: datetime.datetime):
    result = []
    tournaments_fp = get_tournaments()
    n_tournaments = int(tournaments_fp["total_items"])
    tournaments = tournaments_fp["items"]
    for i in range(1, math.ceil(n_tournaments / 1000) + 1):
        tournaments.extend(get_tournaments(i)["items"])
    for t in tournaments:
        try:
            t_end = date_parse(t["date_end"])
        except ValueError:
            continue
        if t_end < release_date and t_end >= release_date - datetime.timedelta(days=7) and t['type_name'] in {'Обычный', 'Синхрон'}:
            result.append(t)
    return result

In [92]:
tournaments_for_release = get_tournaments_for_release(datetime.datetime(2019, 12, 5))

In [95]:
tournaments_results_for_release = {t['idtournament']: get_tournament_results_by_id(t['idtournament']) 
                                   for t in tournaments_for_release}

In [98]:
with open('tournament_results.json', 'w') as f:
    json.dump(tournaments_results_for_release, f)

In [107]:
def calc_rg(tournament_results, team_release, player_release):
    """
    mock function for now
    returns np.array of rgs
    """
    return np.array([int(t['tech_rating_rg']) for t in tournament_results])

In [111]:
def calc_c(ratings):
    ratings[::-1].sort()
    return 2300 / ratings[:15].dot(2.**np.arange(0, -15, -1))

In [126]:
%%timeit
calculate_bonus_predictions(calc_rg(tournaments_results_for_release['5752'], 0, 0), c=0.1194)

1.81 ms ± 240 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
def calculate_changes(team_release, player_release, tournament_results):
    """
    returns Counter({team_id: change}) based on the tournament results
    """
    return Counter()

In [61]:
def calculate_rating_release(team_release: Counter, player_release, tournaments):
    total_changes = Counter()
    for tournament in tournaments:
        total_changes += calculate_changes(team_release, player_release, tournament) 
    return team_release + total_changes

In [63]:
calculate_rating_release(Counter(), Counter(), [])

Counter()

In [60]:
t_one = Counter({'a': 2, 'b': -3})
t_two = Counter({'b': 5, 'c': 4})
t_one + t_two

Counter({'a': 2, 'b': 2, 'c': 4})